In [1]:
import numpy as np
from sklearn.base import BaseEstimator

In [2]:
data_input = np.sin(np.arange(0,stop=360)*6*np.pi/360)

In [3]:
number_of_periods = len(data_input)
periods_to_fit = 180
periods_to_predict = 60
spread = 7  # number of periods to skip to run succesive forecasts

total_number_of_tests = int((number_of_periods - periods_to_fit - periods_to_predict) / spread)

In [4]:
class NaiveLV(BaseEstimator):
    def __init__(self, lag):
        self.lag = lag
    
    def fit(self, X, N):
        self.N = N
        self.X = X
        length = len(X)
        self.forecast = np.zeros((length+self.N,))

        if length < self.lag:
            print('Naive LV: Not enough data')
            return False

        for i in range(0, length+ self.N):
            if i >= self.lag:
                # ex-post
                if i <= length + self.lag - 1:
                    self.forecast[i] = X[i - self.lag]
                # ex-ante
                else:
                    self.forecast[i] = self.forecast[i - 1]

        return self

    
    def predict(self, first_date):
        return self.forecast[first_date:]


In [5]:
naive_lv = NaiveLV(7)
wapes = []

for iteration in range(total_number_of_tests):
    start_fit = iteration*spread
    end_fit = start_fit + periods_to_fit
    end_predict = end_fit + 1 + periods_to_predict
    
    X = data_input[start_fit:end_fit]
    y = data_input[end_fit+1:end_predict]
    
    y_predict = naive_lv.fit(X, periods_to_predict).predict(periods_to_fit)
    wape = 100 * np.sum(abs(y - y_predict)) / np.sum(y_predict)
    print(wape)
    wapes.append(wape)

1004.6781991496473
-155.69918486605604
-74.53764551680847
-77.30475437878515
-96.0172759371218
-118.45398080474652
-145.77037454701662
-190.28948811650457
-326.9738831805186
656.020563424303
97.35338319563493
72.14428343396321
84.43155917716449
105.22365628682972
129.21725349692798
161.2720474902715
225.36156771038634


In [6]:
print(np.mean(wapes))

79.45034858926886
